# MarketLens – AI-Powered Stock Forecasting App
This notebook demonstrates a stock market forecasting model using LSTM.
- Real-time data is fetched using the Alpha Vantage API
- We preprocess the data for time series modeling
- Build and train an LSTM model
- Visualize predictions with confidence intervals

**Tech Stack**: Python, TensorFlow, Alpha Vantage API, Matplotlib, Scikit-learn


In [ ]:
# Install dependencies (uncomment if running in new environment)
# !pip install yfinance numpy pandas matplotlib scikit-learn tensorflow


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Parameters
TICKER = 'AAPL'
START_DATE = '2015-01-01'
END_DATE = '2023-01-01'


In [ ]:
# Fetch data
df = yf.download(TICKER, start=START_DATE, end=END_DATE)
df = df[['Close']]
df.head()

In [ ]:
# Scale and prepare sequences
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

X = []
y = []
sequence_length = 60
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])
    y.append(scaled_data[i])

X, y = np.array(X), np.array(y)
print(X.shape, y.shape)

In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=10, batch_size=32)

In [ ]:
# Predict and visualize
predicted = model.predict(X)
predicted = scaler.inverse_transform(predicted)
real = scaler.inverse_transform(y.reshape(-1, 1))

plt.figure(figsize=(14,6))
plt.plot(real, label='Actual')
plt.plot(predicted, label='Predicted')
plt.title(f'{TICKER} Price Forecast')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.show()